In [ ]:
import subprocess
import os

# Define file paths
input_video = "/content/extracted_audio.mp4"  # This is your uploaded video file
output_audio = "/content/extracted_audio.mp3"  # Output file in Colab storage change your path

# Check if video file exists
if not os.path.exists(input_video):
    raise FileNotFoundError(f"❌ Input video file not found: {input_video}")

# Extract audio using ffmpeg
try:
    result = subprocess.run([
        'ffmpeg', '-i', input_video,
        '-q:a', '0', '-map', 'a', output_audio,
        '-y'  # Overwrite existing file if needed
    ], capture_output=True, text=True)

    # Print ffmpeg logs (optional for debugging)
    print(result.stdout)
    print(result.stderr)

    # Confirm if the file was created successfully
    if os.path.exists(output_audio):
        print(f"✅ Audio successfully extracted and saved to: {output_audio}")
    else:
        print(f"❌ Failed to extract audio.")
except Exception as e:
    print(f"⚠️ Error during audio extraction: {e}")

# List files in /content/ to confirm
!ls -lh /content/



ffmpeg version 4.4.2-0ubuntu0.22.04.1 Copyright (c) 2000-2021 the FFmpeg developers
  built with gcc 11 (Ubuntu 11.2.0-19ubuntu1)
  configuration: --prefix=/usr --extra-version=0ubuntu0.22.04.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libdav1d --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librabbitmq --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libsrt --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvidstab --enable-libvorbis --enable-libvpx --ena

In [ ]:
# Import required libraries
import whisperx

# Set device (Change to "cuda" if using GPU)
device = "cpu"

# Load the WhisperX model (Choose: tiny, base, small, medium, large, large-v2)
model = whisperx.load_model("medium", device=device, compute_type="float32")

# Define the path to your uploaded audio file
audio_file = "/content/extracted_audio.mp3"

# Transcribe the audio
transcription = model.transcribe(audio_file)

# Print the transcription result
print("Transcription:", transcription)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/2.26k [00:00<?, ?B/s]

vocabulary.txt:   0%|          | 0.00/460k [00:00<?, ?B/s]

model.bin:   0%|          | 0.00/1.53G [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.20M [00:00<?, ?B/s]

No language specified, language will be first be detected for each audio file (increases inference time).
>>Performing voice activity detection using Pyannote...


INFO:pytorch_lightning.utilities.migration.utils:Lightning automatically upgraded your loaded checkpoint from v1.5.4 to v2.5.0.post0. To apply the upgrade to your files permanently, run `python -m pytorch_lightning.utilities.upgrade_checkpoint ../usr/local/lib/python3.11/dist-packages/whisperx/assets/pytorch_model.bin`


Model was trained with pyannote.audio 0.0.1, yours is 3.3.2. Bad things might happen unless you revert pyannote.audio to 0.x.
Model was trained with torch 1.10.0+cu102, yours is 2.5.1+cu124. Bad things might happen unless you revert torch to 1.x.
Detected language: en (0.99) in first 30s of audio...
Transcription: {'segments': [{'text': " Thank you very much. This is the meeting at the RM of Springfield. The meeting agenda is a committee of the whole on January the 14th of 2025 at exactly 1 p.m. I'm going to call the meeting to order. I'd like to do some introductions as well, please. We have Roxanne and Christy on the front desk there handling the Wi-Fi component and we have our CAO Colleen Draper.", 'start': 2.933, 'end': 31.537}, {'text': " Myself, I'm Mayor Patrick Katarian for the arm of Springfield. In descending order is Deputy Mayor Glenn Fuel, also Ward 1. Next will be Councillor Ward 2 is Andy Kozinski. Next is, do we know if Mark's here? Mark is not here right now, but if he

In [ ]:
from gpt4all import GPT4All
import re
import json
from trello import TrelloClient
import os



# Extract full transcript text from WhisperX `transcription`
final_transcript = "\n".join(segment['text'] for segment in transcription['segments'])

# ============================================
# Load GPT4All Model (Ensure model file exists in your environment)
gpt_model = GPT4All("mistral-7b-instruct-v0.1.Q4_0.gguf")

# ============================================
# Define GPT4All Prompt for Meeting Analysis
input_text = f"""
You are an expert meeting assistant. Analyze the following meeting transcript and extract the key discussion points, decisions made, and action items.

### Transcript ###
{final_transcript}

### Expected Output ###
Give the summarized meeting report first, followed by:

- **Key Discussion Points**:
- **Decisions Made**:
- **Action Items**:
"""

# ============================================
# Generate Structured Output using GPT4All
with gpt_model.chat_session():
    gpt_output = gpt_model.generate(input_text, max_tokens=800, temp=0.7)

# ============================================
# Parse GPT Output into Lists
summary_report = []
discussion_points = []
decisions_made = []
action_items = []
def parse_gpt4all_summary(gpt_output):
    """
    Parse the GPT4All output into structured lists (summary report, discussion points, decisions made, action items)
    """

    sections = re.split(r"- \*\*(.*?)\*\*:", gpt_output)


    current_section = None

    for i, section in enumerate(sections):
        section = section.strip()

        if i == 0:
            summary_report = [line.strip("- ") for line in section.split("\n") if line.strip()]
        elif section.lower().startswith("key discussion points"):
            current_section = discussion_points
        elif section.lower().startswith("decisions made"):
            current_section = decisions_made
        elif section.lower().startswith("action items"):
            current_section = action_items
        elif current_section is not None:
            items = [line.strip("- ") for line in section.split("\n") if line.strip()]
            current_section.extend(items)

        # Initialize Trello client
        client = TrelloClient(
            api_key='giveYourApiKeyHere',
            token='giveYourTokenHere'
        )


        # Create a new board
        board_name = "kkrn10000"
        board = client.add_board(board_name)
        print(f"Created board: {board.name} (ID: {board.id})")

        # Add lists to the board
        action_items_list = board.add_list("Action Items")
        print(f"Created list: {action_items_list.name} (ID: {action_items_list.id})")

        discussion_points_list = board.add_list("Discussion Points")
        print(f"Created list: {discussion_points_list.name} (ID: {discussion_points_list.id})")

        summary_report_list = board.add_list("Summary Report")
        print(f"Created list: {summary_report_list.name} (ID: {summary_report_list.id})")

        decisions_made_list = board.add_list("Decisions Made")
        print(f"Created list: {decisions_made_list.name} (ID: {decisions_made_list.id})")

        # Add cards to the Action Items list
        for item in action_items:
            card = action_items_list.add_card(item)
            print(f"Added card to Action Items: {card.name}")

        # Add cards to the Discussion Points list
        for point in discussion_points:
            card = discussion_points_list.add_card(point)
            print(f"Added card to Discussion Points: {card.name}")

        # Add cards to the Summary Report list
        for report in summary_report:
            card = summary_report_list.add_card(report)
            print(f"Added card to Summary Report: {card.name}")

        # Add cards to the Decisions Made list
        for decision in decisions_made:
            card = decisions_made_list.add_card(decision)
            print(f"Added card to Decisions Made: {card.name}")

        print(f"Board '{board_name}' created successfully with lists and cards!")


    return {
        "summary_report": summary_report,
        "discussion_points": discussion_points,
        "decisions_made": decisions_made,
        "action_items": action_items
    }

# Parse GPT output into structured lists
parsed_data = parse_gpt4all_summary(gpt_output)



Created board: kkrn10000 (ID: 67c9f1f263783d759a2bc87e)
Created list: Action Items (ID: 67c9f1f29eb11db58d1f1e6e)
Created list: Discussion Points (ID: 67c9f1f3ee561d197168e610)
Created list: Summary Report (ID: 67c9f1f334c86b69b4278d58)
Created list: Decisions Made (ID: 67c9f1f313b7c54b45b5761a)
Added card to Summary Report: Summarized Meeting Report:
Added card to Summary Report: The meeting was held at RM of Springfield on January 14th, 2025. The agenda included a committee of the whole discussion. Participants in the meeting were Mayor Patrick Katarian, Deputy Mayor Glenn Fuel, Ward 1 Councillor Andy Kozinski, Ward 2 Councillor Melinda Warren, and Roxanne and Christy from the front desk handling Wi-Fi component. The CAO Colleen Draper was also present.
Added card to Summary Report: Key Discussion Points:
Added card to Summary Report: * Introduction of participants
Added card to Summary Report: * Approval of agenda with item number three discussed first (committee of the whole)
Added